In [57]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import time
import json
import sys
from concurrent.futures import ThreadPoolExecutor
import re
from tqdm import tqdm # Progress bar for visualizing data cleaning progress
from fuzzywuzzy import fuzz

# Defined global variables
csvFile = "./truncatedData.csv";
cikFile = "./cik-lookup-data.txt";
minDate = datetime(2001, 1, 1);
formTypes = ["PREM14A", "S-4", "SC 14D9", "SC TO-T"];
mainIndex = 2;

# Read the CSV file and extract the date & both merging companies (index base)
filedDate = pd.read_csv(csvFile, header=None).iloc[:, 1].tolist();
companyAList = pd.read_csv(csvFile, header=None).iloc[:, 2].tolist();
companyBList = pd.read_csv(csvFile, header=None).iloc[:, 3].tolist();

In [58]:
# Acquire the constraint of a given date.
# Pad 2 months backward and forward for constraint.
def getDateConstraints(date):
    originalDate = datetime.strptime(date, "%m/%d/%Y");

    # Define the lower-bound date
    lbMonth = originalDate.month - 2;
    if (lbMonth <= 0): # Case: Wrap to previous year
        lbMonth += 12;
        lbYear = originalDate.year - 1;
    else: # Case: Still on current year
        lbYear = originalDate.year;

    # Construct lower-bound date
    try:
        lowerBoundDate = originalDate.replace(year=lbYear, month=lbMonth);
    except ValueError: # Catch potential error i.e. feb. 30 not existing
        lowerBoundDate = originalDate.replace(year=lbYear, month=lbMonth, day=1);

    # Ensure the new date does not go below the minimum date
    if (lowerBoundDate < minDate):
        lowerBoundDate = minDate;

    
    # Define the upper-bound date
    ubMonth = originalDate.month + 2;
    if (ubMonth > 12): # Case: Wrap to next year
        ubMonth -= 12;
        ubYear = originalDate.year + 1;
    else: # Case: Still on current year
        ubYear = originalDate.year;

    # Construct upper-bound date
    try:
        upperBoundDate = originalDate.replace(year=ubYear, month=ubMonth);
    except ValueError: # Catch potential error i.e. feb. 30 not existing
        upperBoundDate = originalDate.replace(year=ubYear, month=ubMonth + 1, day=1);

    return [lowerBoundDate, upperBoundDate];

In [59]:
# Get the list of CIKs for the merging companies
def getCIKS(searchCompany, pairCompany, dateLB, dateUB, formTypes):
    restructName = searchCompany.replace(" ", "%20");
    
    url = f"https://efts.sec.gov/LATEST/search-index?q={restructName}&dateRange=custom&category=custom&startdt={dateLB}&enddt={dateUB}&forms={formTypes}";

    # Create a request that mimics browser activity
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "Referer": "https://www.sec.gov/"
    }

    # Request the search query & acquire the DOM elements
    response = requests.get(url, headers=headers);
    if (response.status_code != 200):
        print("FATAL: getDocumentJson response yielded an error!");
        sys.exit(response.status_code);
    
    data = response.json();
    totalValue = data["hits"]["total"]["value"];

    if (totalValue <= 0):
        return None;
    
    # Formulate the list of entities for CIK lookup
    entityList = [];
    for entities in data["aggregations"]["entity_filter"]["buckets"]:
        entityList.append(entities["key"]);

    # Acquire the CIK for the given company using combined fuzzy matching techniques
    threshold = 90
    filteredMatch = [
        entity for entity in entityList if fuzz.partial_ratio(pairCompany.lower(), entity.lower()) > threshold
    ]
    
    # Extract the CIK from the filtered match
    cikList = [];
    for entity in filteredMatch:
        cikList.append(re.search(r'\(CIK (\d+)\)', entity).group(1));

    return cikList if cikList else None;

In [60]:
# Acquire all the json documents for the given companies with CIK filter
def getCIKDocumentJson(searchCompany, pairCompany, dateLB, dateUB, formTypes):
    # Remove parantheses content from the company names
    searchCompany = re.sub(r'\(.*\)', '', searchCompany).strip();
    pairCompany = re.sub(r'\(.*\)', '', pairCompany).strip();

    # We will try and acquire the cikList for the first company;
    # If the cikList is None, we will try and acquire the cikList for the second company.
    cikList = getCIKS(searchCompany, pairCompany, dateLB, dateUB, formTypes);
    if (cikList == None):
        cikList = getCIKS(pairCompany, searchCompany, dateLB, dateUB, formTypes);
    
    if (cikList == None):
        return None;

    """
        - Fetch data for each CIK concurrently
        - We do not need to verify if the hit returns nothing as if the entity is not found,
            the CIK will not be present in the list.
    """
    restructName = searchCompany.replace(" ", "%20");

    urls = [f"https://efts.sec.gov/LATEST/search-index?q={restructName}&dateRange=custom&category=custom&startdt={dateLB}&enddt={dateUB}&forms={formTypes}&filter_ciks={cik}" for cik in cikList];
    
    # Create a request that mimics browser activity
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "Referer": "https://www.sec.gov/"
    }

    # Fetch the json data for each CIK
    if len(urls) == 1: # Case: Single URL; no threads required
        response = requests.get(urls[0], headers=headers);
        if (response.status_code != 200):
            print("FATAL: getDocumentJson response yielded an error!");
            sys.exit(response.status_code);
        
        result = response.json();
        mergedHits = result["hits"]["hits"] if result and "hits" in result and "hits" in result["hits"] else [];
    else: # Case: Multiple URLs; use threads for concurrent fetching
        with ThreadPoolExecutor() as executor:
            results = list(executor.map(lambda url: requests.get(url, headers=headers), urls));
        
        # Merge the results into a single list
        mergedHits = [];
        for response in results:
            if (response.status_code != 200):
                print("FATAL: getDocumentJson response yielded an error!");
                sys.exit(response.status_code);

            result = response.json();
            if result and "hits" in result and "hits" in result["hits"]:
                mergedHits.extend(result["hits"]["hits"]);

    return mergedHits if mergedHits else None;

In [61]:
"""
    - No documents were found associated with the CIKs.
    - We will let fuzzy match determine if the company is present in the document.
        - Return a truncated list of documents for both companies without the cik filter.
    - Basically throwing a dart at the board and hoping it hits the target if no cik filtering is found.
"""
def getDocumentJson(searchCompany, pairCompany, dateLB, dateUB, formTypes):
    # Remove parantheses content from the company names
    searchCompany = re.sub(r'\(.*\)', '', searchCompany).strip();
    pairCompany = re.sub(r'\(.*\)', '', pairCompany).strip();

    restructSearch = searchCompany.replace(" ", "%20");
    restructPair = pairCompany.replace(" ", "%20");

    urls = [
        f"https://efts.sec.gov/LATEST/search-index?q={restructSearch}&dateRange=custom&category=custom&startdt={dateLB}&enddt={dateUB}&forms={formTypes}",
        f"https://efts.sec.gov/LATEST/search-index?q={restructPair}&dateRange=custom&category=custom&startdt={dateLB}&enddt={dateUB}&forms={formTypes}"
    ];
    
    # Create a request that mimics browser activity
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "Referer": "https://www.sec.gov/"
    }

    # Fetch the json data for each company using threads for concurrent fetching
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda url: requests.get(url, headers=headers), urls));
    
    # Merge the results into a single list
    mergedHits = [];
    for response in results:
        if (response.status_code != 200):
            print("FATAL: getDocumentJson response yielded an error!");
            sys.exit(response.status_code);

        result = response.json();
        if result and "hits" in result and "hits" in result["hits"]:
            mergedHits.extend(result["hits"]["hits"]);

    return mergedHits if mergedHits else None;

In [62]:
# Formulate the source document links from the search result json
def getSourceLinks(documentJson):
    # Formulate all source document file links
    sourceLinks = [];

    # Iterate through each json object and construct the source document file links
    for document in documentJson:
        try:
            # Get the CIK id or if there is multiple, then acquire the last one
            validatedCik = None;
            ciks = document["_source"]["ciks"];
            if ciks:
                lastCIK = ciks[-1];
                validatedCik = lastCIK;

            # Remove leading zeros from the CIK
            validatedCik.lstrip('0');
        
            # Acquire normal adsh & adsh without the "-" character
            adsh = document["_source"]["adsh"];
            truncatedADSH = document["_source"]["adsh"].replace("-", "");
            
            sourceLinks.append(f"https://www.sec.gov/Archives/edgar/data/{validatedCik}/{truncatedADSH}/{adsh}.txt");
        except KeyError as e:
            print(f"Missing key in document: {e}, result: {document}");
            continue; # Skip the document if there is a missing key; logged for further investigation

    return sourceLinks;

In [63]:
constraintDates = getDateConstraints(filedDate[mainIndex]);
lbDate, ubDate = constraintDates;
restructLB = f"{lbDate.year}-{lbDate.month:02}-{lbDate.day:02}";
restructUB = f"{ubDate.year}-{ubDate.month:02}-{ubDate.day:02}";
restructForms = "%2C".join(formTypes).replace(" ", "%20");

# Find the documents with CIK filtering
results = getCIKDocumentJson(companyAList[mainIndex], companyBList[mainIndex], restructLB, restructUB, restructForms);
if (results == None): # Acquire all documents within our guess
    results = getDocumentJson(companyAList[mainIndex], companyBList[mainIndex], restructLB, restructUB, restructForms);

# No documents found for our 2 companies
if (results == None):
    print("No documents found for: ", companyAList[mainIndex], " & ", companyBList[mainIndex]);
    sys.exit(1);

# Extract the source document links
sourceLinks = getSourceLinks(results);
print("\n".join(map(str, sourceLinks)))

# for index in range(100):
#     constraintDates = getDateConstraints(filedDate[index]);
#     lbDate, ubDate = constraintDates;
#     restructLB = f"{lbDate.year}-{lbDate.month:02}-{lbDate.day:02}";
#     restructUB = f"{ubDate.year}-{ubDate.month:02}-{ubDate.day:02}";
#     restructForms = "%2C".join(formTypes).replace(" ", "%20");

#     results = getDocumentJson(companyAList[index], companyBList[index], restructLB, restructUB, restructForms);
#     if (results == None):
#         print("Result is none for index: ", index, "; ", companyAList[index], " & ", companyBList[index]);

#     # time.sleep(0.25);

https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000120/0000950170-01-000120.txt
https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000120/0000950170-01-000120.txt
https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000120/0000950170-01-000120.txt
https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000170/0000950170-01-000170.txt
https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000120/0000950170-01-000120.txt
https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000170/0000950170-01-000170.txt
https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000120/0000950170-01-000120.txt
https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000120/0000950170-01-000120.txt
https://www.sec.gov/Archives/edgar/data/0001110189/000095015901000024/0000950159-01-000024.txt
https://www.sec.gov/Archives/edgar/data/0001123337/000095017001000120/0000950170-01-000120.txt
https://www.sec.gov/Archives/edgar/data/0001123337